In [211]:
import sklearn as skl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.io
from sklearn.utils.validation import check_is_fitted

In [210]:
X = np.array([[3, 4, -8.904, -5.974], [3, 4, -3.042, -6.077], [5, 2, -2.940, -6.065], [3, 2, -6.863, -5.992], [4, 5, -3.132, -4.162], [4, 4, -1.138, -4.087], [3, 3, -6.168, -6.091], [3, 4, -3.888, -5.187], [4, 4, -8.218, -5.992], [4, 4, -7.811, -5.977], [3, 4, -1.775, -5.698], [3, 3, -2.126, -6.102]])

count = 0
n_clusters = 2
alpha = 0.99
T_min = 0.001
delta = 3
samples, n_features = X.shape
C_x = np.zeros((n_features, n_features))
for i in range(samples):
    C_x = C_x + (1 / samples) * np.transpose(X[i, :]) * X[i, :]
eigval, eigvec = np.linalg.eig(C_x)
T = float(2 * np.amax(eigval) + 1)
T = T / alpha
K = 1
cluster_centers_ = np.zeros((2, n_features))
cluster_centers_[0, :] = np.sum(X, axis=0) / samples
cluster_probabs_ = np.zeros((samples, 2))
cluster_probabs_[:, 0] = np.ones((samples, 2))[:, 0]
prob_centroid = np.zeros((2, 1))
prob_centroid[0] = 1

while (T > T_min):
    T = T * alpha
    elements = np.zeros((samples, K + 1))
    n = 0
    temp_prob_centroid = np.copy(prob_centroid)
    temp_cluster_centers_ = np.copy(cluster_centers_)
    temp_cluster_probabs_ = np.copy(cluster_probabs_)
        
    while (n == 0 or (temp_prob_centroid - prob_centroid > 0.01).any() or (temp_cluster_centers_ - cluster_centers_ > 0.01).any() or (temp_cluster_probabs_ - cluster_probabs_ > 0.01).any()):
        prob_centroid = np.copy(temp_prob_centroid)
        cluster_centers_ = np.copy(temp_cluster_centers_)
        cluster_probabs_ = np.copy(temp_cluster_probabs_)
        n = 1
        for i in range(K):
            elements[:, i] = temp_prob_centroid[i] * np.exp(- (np.transpose(np.linalg.norm(X - temp_cluster_centers_[i, :], axis=1))) / T)
        for i in range(K):
            denom = np.transpose(np.sum(elements, axis=1))
            temp_cluster_probabs_[:, i] = np.divide(elements[:, i], denom)
            temp_prob_centroid[i] = float(np.sum(temp_cluster_probabs_[:, i])) / samples
            temp_cluster_centers_[i, :] = ((np.transpose(np.matmul(np.transpose(X), temp_cluster_probabs_[:, i]))) / np.sum(temp_cluster_probabs_[:, i]))
            Kold = K
    if (Kold < n_clusters):
        for i in range(Kold):
            if (Kold < n_clusters):
                C_x = np.zeros((n_features, n_features))
                for j in range(samples):
                    C_x = C_x + (cluster_probabs_[j, i] / (samples * prob_centroid[i])) * np.transpose(np.subtract(X[j, :], cluster_centers_[i, :])) * (np.subtract(X[j, :], cluster_centers_[i, :]))
                eigval, eigvec = np.linalg.eig(C_x)
                to_compare = 2 * np.amax(eigval)
                if (K == n_clusters):
                    break
                if (T < to_compare):
                    if K == 1:
                        cluster_probabs_[:, i] = cluster_probabs_[:, i] / 2
                        cluster_probabs_[:, i + 1] = cluster_probabs_[:, i]
                        cluster_centers_[i + 1, :] = cluster_centers_[i, :] + delta
                        prob_centroid[i] = prob_centroid[i] / 2
                        prob_centroid[i + 1] = prob_centroid[i]
                        K = K + 1
                    else:
                        cluster_probabs_[:, i] = cluster_probabs_[:, i] / 2
                        cluster_probabs_ = np.append(cluster_probabs_, cluster_probabs_[:, i:i+1], axis=1)
                        cluster_centers_ = np.append(cluster_centers_, ([cluster_centers_[i, :] + delta]), axis=0)
                        prob_centroid[i] = prob_centroid[i] / 2
                        prob_centroid = np.append(prob_centroid, prob_centroid[i])
                        K = K + 1

C:\Users\edoar\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in true_divide
C:\Users\edoar\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in greater


In [214]:
print('Class membership probability\n', cluster_probabs_)
print('Probability of class centroid\n', prob_centroid)
print('Class centroids\n', cluster_centers_)

Class membership probability
 [[9.99975528e-01 2.44721132e-05]
 [3.18624535e-04 9.99681375e-01]
 [3.39061355e-03 9.96609386e-01]
 [9.99699685e-01 3.00314748e-04]
 [8.79353998e-04 9.99120646e-01]
 [2.61983822e-05 9.99973802e-01]
 [9.99378447e-01 6.21553017e-04]
 [5.61343880e-03 9.94386561e-01]
 [9.99970441e-01 2.95589164e-05]
 [9.99965641e-01 3.43594102e-05]
 [2.44931641e-05 9.99975507e-01]
 [8.32566320e-05 9.99916743e-01]]
Probability of class centroid
 [[0.41744381]
 [0.58255619]]
Class centroids
 [[ 3.40077707  3.40016619 -7.58448248 -6.00398525]
 [ 3.57110044  3.71458589 -2.57655501 -5.33969697]]
